In [2]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

seed = np.random.randint(2**32-1)

task = VideogameReviewsTask(seed=seed)

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

In [4]:
model = task.fit_baseline_model(task.train_data, task.train_labels)

In [6]:
stress_test = SchemaStresstest()


In [9]:
results = stress_test.run(task, model, schema, num_corruptions=10, performance_threshold=.03)

Scaling: {'column': 'star_rating', 'fraction': 0.61}


In [10]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,"Scaling: {'column': 'star_rating', 'fraction':...",FN,{},0.791215,0.575134
